In [10]:
import pandas as pd
from Scripts.dirty_accuracy import injection

In [11]:
german = pd.read_csv("datasets/german.csv", sep=',')
german_list = injection(df_pandas=german, seed=10, name='german', name_class='Risk')
german_list.append(german)
german_list.reverse()

saved german-accuracy50%
saved german-accuracy60%
saved german-accuracy70%
saved german-accuracy80%
saved german-accuracy90%


18424

german_list[0] is the cleanest (accuracy 100%), german_list[5] is the dirtiest (accuracy 50%)

,Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose,Risk
0,67.000000,male,-21.765475,own,little,little,1169.000000,6.000000,radio/TV,good
1,182.120528,female,2.000000,own,little,moderate,5951.000000,510.231018,radio/TV,bad
2,49.000000,male,23.605008,own,little,little,2096.000000,12.000000,education,good
3,45.000000,male,2.000000,free,little,little,-70759.050483,419.751074,furniture/equipment,good
4,614.832117,male,9.776082,free,little,little,-177555.737614,24.000000,car,bad
